In [1]:
import numpy as np


# Alternative definition of the DFT, its inverse and adjoint

## Forward transform

In the paper, an alternative definition of the DFT is used. Let $n \in \mathbb{N}$ and $x \in \mathbb{C}^n$. We denote by $n/2$ the quotient of the division of $n$ by $2$ (which corresponds to ``n//2`` in Python). We define:
$$
\mathcal{D}(n) = 
\begin{cases}
\llbracket - n/2, n/2 - 1\rrbracket &\text{ if } n \text{ is even} \\
\llbracket - n/2, n/2 \rrbracket &\text{ else}
\end{cases}
$$

Then, the new DFT of a signal $x : \mathcal{D}(n) \to \mathbb{C}$ is defined as:
$$
\forall k \in \mathcal{D}(n), \, \widehat{x}(k) = \sum_{u \in \mathcal{D}(n)} x(u) \exp\left(-2i \pi \dfrac{k u}{n} \right)
$$

A fast computation of $\widehat{x}$ can be obtained using the FFT alongside with ``fftshift`` and ``ifftshift``.

We implement and test it here.

### In 1D:

Naive version, used to test the other one:

In [2]:
def naive_fourier(x):
    n = len(x)
    q_n, r_n = divmod(n, 2)

    u = np.arange(-q_n, q_n + r_n)
    k = np.arange(-q_n, q_n + r_n)

    ku = np.exp(-2j * np.pi * np.einsum("k,u->ku", k, u) / n)
    res = np.einsum("u,ku->k", x, ku)

    return res


Fast version:

In [3]:
def new_fft(x, axis=-1):
    return np.fft.fftshift(np.fft.fft(np.fft.ifftshift(x, axes=axis), axis=axis), axes=axis)


Tests:

In [4]:
x = np.random.rand(10)
print(np.max(np.abs(naive_fourier(x) - new_fft(x))))
x = np.random.rand(11)
print(np.max(np.abs(naive_fourier(x) - new_fft(x))))


1.2272864783562909e-15
1.1957467920563633e-15


### In 2D:

An image of size $(n, m)$ is represented by a function $I : \mathcal{D}(n) \times \mathcal{D}(m) \to \mathbb{C}$. Its DFT is:
$$
\forall (l,k) \in \mathcal{D}(n) \times \mathcal{D}(m), \, \widehat{I}(l, k) = \sum_{u \in \mathcal{D}(n)} \sum_{v \in \mathcal{D}(m)} I(u, v) \exp\left(-2i \pi \frac{l u}{n} \right) \exp\left(-2i \pi \frac{k v}{m} \right)
$$

In [5]:
def naive_fourier_2D(a):
    n, m = a.shape

    q_n, r_n = divmod(n, 2)
    u = np.arange(-q_n, q_n + r_n)
    l = np.arange(-q_n, q_n + r_n)

    q_m, r_m = divmod(m, 2)
    v = np.arange(-q_m, q_m + r_m)
    k = np.arange(-q_m, q_m + r_m)

    lu = np.exp(-2j * np.pi * np.einsum("l,u->lu", l, u) / n)
    kv = np.exp(-2j * np.pi * np.einsum("k,v->kv", k, v) / m)

    res = np.einsum("uv,ku,lv->kl", a, lu, kv)

    return res


In [6]:
def new_fft2(x):
    return np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(x)))


In [7]:
shapes = [(4, 4), (4, 5), (5, 4), (5, 5)]

for shape in shapes:
    a = np.random.random(shape)
    print(np.max(np.abs(naive_fourier_2D(a) - new_fft2(a))))


8.881784197001252e-16
7.947987303456224e-16
3.552713678800501e-15
8.671119018262734e-16


## Inverse DFT

The definition of the inverse DFT is:
$$
\forall k \in \mathcal{D}(n), \, \mathcal{F}^{-1}(y)(k) = \sum_{u \in \mathcal{D}(n)} y(u) \exp\left(2i \pi \dfrac{k u}{n} \right)
$$

### In 1D:

In [8]:
def new_ifft(x, axis=-1):
    return np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(x, axes=axis), axis=axis), axes=axis)


In [9]:
x = np.random.rand(10)
print(np.max(np.abs(new_ifft(new_fft(x)) - x)))
x = np.random.rand(11)
print(np.max(np.abs(new_ifft(new_fft(x)) - x)))


1.123827244794695e-16
2.220446049250313e-16


In [10]:
x = np.random.rand(10)
print(np.max(np.abs(new_fft(new_ifft(x)) - x)))
x = np.random.rand(11)
print(np.max(np.abs(new_fft(new_ifft(x)) - x)))


2.220658297631543e-16
1.1102230246251565e-16


### In 2D:

In [11]:
def new_ifft2(a: np.ndarray) -> np.ndarray:
    return np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(a)))


In [12]:
x = np.random.rand(10, 10)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))
x = np.random.rand(10, 11)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))
x = np.random.rand(11, 10)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))
x = np.random.rand(11, 11)
print(np.max(np.abs(new_ifft2(new_fft2(x)) - x)))


3.344583420624203e-16
3.352055111865361e-16
3.364808191022983e-16
4.440892098500626e-16


## Adjoints

In [13]:
def matrix_dot(a, b):
    return np.sum(np.conjugate(a) * b)


In [14]:
def adj_new_fft(x, axis=-1):
    return new_ifft(x, axis) * np.shape(x)[axis]


def adj_new_ifft(x, axis=-1):
    return new_fft(x, axis) / np.shape(x)[axis]


def adj_new_fft2(a):
    return new_ifft2(a) * np.size(a)


def adj_new_ifft2(a):
    return new_fft2(a) / np.size(a)


In [15]:
x = np.random.rand(10)
y = np.random.rand(10)
print(np.abs(np.vdot(new_fft(x), y) - np.vdot(x, adj_new_fft(y))))
x = np.random.rand(11)
y = np.random.rand(11)
print(np.abs(np.vdot(new_fft(x), y) - np.vdot(x, adj_new_fft(y))))


9.155133597044475e-16
9.930136612989092e-16


In [16]:
x = np.random.rand(10)
y = np.random.rand(10)
print(np.abs(np.vdot(new_ifft(x), y) - np.vdot(x, adj_new_ifft(y))))
x = np.random.rand(11)
y = np.random.rand(11)
print(np.abs(np.vdot(new_ifft(x), y) - np.vdot(x, adj_new_ifft(y))))


5.721958498152797e-17
3.469446951953614e-18


In [17]:
shapes = [(4, 4), (4, 5), (5, 4), (5, 5)]
err_list = []

for shape in shapes:
    x = np.random.random(shape)
    y = np.random.random(shape)
    print(np.abs(matrix_dot(new_fft2(x), y) - matrix_dot(x, adj_new_fft2(y))))


4.440892098500626e-16
1.784146017590271e-15
1.790180836524724e-15
3.580361673049448e-15


In [40]:
shapes = [(4, 4), (4, 5), (5, 4), (11, 13)]
err_list = []

for shape in shapes:
    x = np.random.random(shape)
    y = np.random.random(shape)
    print(np.isclose(np.vdot(new_ifft2(x), y), np.vdot(x, adj_new_ifft2(y))))


True
True
True
True
